# Checking lab data

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [ ]:
lab_data = pd.read_csv('../../secret_data/BRICK_lab_data_update_18_03_2024.csv', sep= ';', encoding='latin1')


In [ ]:
lab_data.head(3)

In [ ]:
lab_data['studieID'].unique()

In [ ]:
len(lab_data['studieID'].unique())

In [ ]:
lab_data['studieID'] = lab_data['studieID'].astype(str)
lab_data['studieID'] = lab_data['studieID'].str.replace("-", "_")

In [ ]:
lab_data.head(3)

In [ ]:
lab_data.code_code_original.unique()

In [ ]:
codes = lab_data.code_code_original.unique().astype(str)
codes

In [ ]:
list_of_data_frames = []
for code in codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = lab_data[lab_data['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    list_of_data_frames.append(frame_name)
    

In [ ]:
list_of_data_frames[1].head(3) 

In [ ]:
list_of_data_frames[1].shape

In [ ]:
# dumped = []
# for frame, next_frame in zip(list_of_data_frames, list_of_data_frames[1:]):
#     lovi = frame.merge(next_frame,on=['studieID'], how='outer')
#     print(lovi.shape)

In [ ]:
len(list_of_data_frames)

In [ ]:
final_df2 = list_of_data_frames[0].merge(list_of_data_frames[1], on=['studieID'], how='outer')

In [ ]:
final_df2 = final_df2.merge(list_of_data_frames[3], on=['studieID'], how='outer').merge(list_of_data_frames[4], on=['studieID'], how='outer')

In [ ]:
final_df2 = final_df2.merge(list_of_data_frames[5], on=['studieID'], how='outer').merge(list_of_data_frames[6], on=['studieID'], how='outer')

In [ ]:
final_df2 = final_df2.merge(
    list_of_data_frames[7], on=['studieID'], how='outer'
).merge(list_of_data_frames[8], on=['studieID'], how='outer'
).merge(list_of_data_frames[9], on=['studieID'], how='outer'
).merge(list_of_data_frames[10], on=['studieID'], how='outer'
).merge(list_of_data_frames[11], on=['studieID'], how='outer'
).merge(list_of_data_frames[12], on=['studieID'], how='outer'
).merge(list_of_data_frames[13], on=['studieID'], how='outer'
).merge(list_of_data_frames[14], on=['studieID'], how='outer'
).merge(list_of_data_frames[15], on=['studieID'], how='outer'
).merge(list_of_data_frames[16], on=['studieID'], how='outer'
).merge(list_of_data_frames[17], on=['studieID'], how='outer'
).merge(list_of_data_frames[18], on=['studieID'], how='outer'
).merge(list_of_data_frames[19], on=['studieID'], how='outer'
).merge(list_of_data_frames[20], on=['studieID'], how='outer'
).merge(list_of_data_frames[21], on=['studieID'], how='outer'
).merge(list_of_data_frames[22], on=['studieID'], how='outer'
).merge(list_of_data_frames[23], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[24], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[25], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[26], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[27], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[28], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[29], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[30], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[31], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[32], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[33], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[34], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[35], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[36], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[37], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[38], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[39], on=['studieID'], how='outer'
       )

In [ ]:
final_df2

In [ ]:
for f in final_df2['studieID']:
    print(f)

In [ ]:
final_df2[final_df2['studieID'] == 'BRICK_055']

In [ ]:
STOP

In [ ]:
final_df2.columns

In [ ]:
from functools import reduce

In [ ]:
final_df = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), list_of_data_frames)

In [ ]:
final_df

In [ ]:
final_df[final_df['studieID'] =='BRICK_001']

In [ ]:
for f in final_df.columns:
    print(f)

In [ ]:
final_df.to_csv('long_lab_data.csv')

In [ ]:
final_df.shape